In [2]:
!unzip /content/drive/MyDrive/dataset.zip
!unzip /content/drive/MyDrive/test.zip

Streaming output truncated to the last 5000 lines.
  inflating: Y/eventroll_7138.npy    
  inflating: Y/eventroll_267.npy     
  inflating: Y/eventroll_4650.npy    
  inflating: Y/eventroll_6044.npy    
  inflating: Y/eventroll_9118.npy    
  inflating: Y/eventroll_7486.npy    
  inflating: Y/eventroll_6278.npy    
  inflating: Y/eventroll_9926.npy    
  inflating: Y/eventroll_8537.npy    
  inflating: Y/eventroll_563.npy     
  inflating: Y/eventroll_855.npy     
  inflating: Y/eventroll_1363.npy    
  inflating: Y/eventroll_3892.npy    
  inflating: Y/eventroll_9989.npy    
  inflating: Y/eventroll_2795.npy    
  inflating: Y/eventroll_436.npy     
  inflating: Y/eventroll_2969.npy    
  inflating: Y/eventroll_7419.npy    
  inflating: Y/eventroll_4523.npy    
  inflating: Y/eventroll_4856.npy    
  inflating: Y/eventroll_2685.npy    
  inflating: Y/eventroll_2840.npy    
  inflating: Y/eventroll_4388.npy    
  inflating: Y/eventroll_6724.npy    
  inflating: Y/eventroll_4114.npy    

In [3]:
import numpy as np
import pandas as pd
import librosa
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import IPython.display as ipd
from tensorflow.keras.optimizers import SGD
from keras import regularizers


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def eventroll_to_multihot_vector(eventroll):
    """
    Parameters
    ----------
    eventroll : np.array
        Eventroll matrix of shape=(11, 1000).
    
    Returns
    -------
    np.array
        A multihot vector of shape=(10,)
    """
    
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    
    # remove silence class:
    return np.array(active_events)

In [5]:
l = os.listdir('/content/X')
l.sort()
filename_data = pd.DataFrame(l , columns = ['filename'])

l = os.listdir('/content/Y')
l.sort()
event_data= pd.DataFrame(l , columns = ['eventname'])


In [6]:
def r_shape(data):
  data = np.reshape(data,(64,1000))
  data = np.reshape(data,(64,1000,1))
  return data
#def r_shape_2(data):
 # data=np.reshape(data,(1000,11))
  #print(data.shape)
  #return data


In [7]:
data=np.zeros((10000,64,1000,1))
labels=np.zeros((10000,11))
for i,row in filename_data.iterrows():
  file_name = os.path.join(os.path.abspath('/content/X'),str(row["filename"]))
  d=np.load(file_name)
  data[i]=r_shape(d)




In [8]:
for i,row in event_data.iterrows():
  file_name = os.path.join(os.path.abspath('/content/Y'),str(row["eventname"]))
  e = np.load(file_name)
  e=eventroll_to_multihot_vector(e)
  labels[i]=e

In [9]:
#x_train, x_test, y_train, y_test = train_test_split(data,labels, test_size=0.20,random_state=42)

In [9]:
x_train=data
y_train=labels
del(data)
del(labels)

In [10]:
input_shape=(64,1000,1)
x_train.shape,y_train.shape

((10000, 64, 1000, 1), (10000, 11))

In [11]:
model =  models.Sequential([
    
                          layers.Conv2D(32 , (3,3),activation = 'relu',padding='valid', input_shape = input_shape),  
                          layers.BatchNormalization(),
                          layers.Conv2D(64, (3,3), activation='relu',padding='valid'),
                          layers.Conv2D(32, (3,3), activation='relu',padding='valid'),
                          layers.MaxPooling2D(2, 2),
                          layers.Flatten(),
                          layers.Dense(16 , activation = 'relu'),
                          layers.BatchNormalization(),
                          layers.Dropout(0.2),
                          layers.Dense(11 , activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = 'acc')
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 998, 32)       320       
                                                                 
 batch_normalization (BatchN  (None, 62, 998, 32)      128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 996, 64)       18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 58, 994, 32)       18464     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 29, 497, 32)      0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 461216)            0

In [12]:
#history = model.fit(x_train, y_train, epochs=15, validation_data= (x_test, y_test),batch_size=32)
history = model.fit(x_train, y_train, epochs=15, batch_size=32)

Epoch 1/15
313/313 [==============================] - 76s 193ms/step - loss: 0.5096 - acc: 0.0887
Epoch 2/15
313/313 [==============================] - 61s 196ms/step - loss: 0.3279 - acc: 0.0956
Epoch 3/15
313/313 [==============================] - 62s 198ms/step - loss: 0.2975 - acc: 0.0974
Epoch 4/15
313/313 [==============================] - 62s 198ms/step - loss: 0.2696 - acc: 0.1005
Epoch 5/15
313/313 [==============================] - 62s 198ms/step - loss: 0.2411 - acc: 0.1074
Epoch 6/15
313/313 [==============================] - 62s 198ms/step - loss: 0.2160 - acc: 0.1215
Epoch 7/15
313/313 [==============================] - 62s 198ms/step - loss: 0.1969 - acc: 0.1337
Epoch 8/15
313/313 [==============================] - 62s 198ms/step - loss: 0.1780 - acc: 0.1418
Epoch 9/15
313/313 [==============================] - 62s 198ms/step - loss: 0.1641 - acc: 0.1512
Epoch 10/15
313/313 [==============================] - 62s 198ms/step - loss: 0.1537 - acc: 0.1627
Epoch 11/15
313/313

In [13]:
del(x_train)
del(y_train)

In [14]:
def r_shape_2(data):
  data = np.reshape(data,(1,64,1000))
  data = np.reshape(data,(1,64,1000,1))
  return data
filename=[]
filename = os.listdir('/content/test/X')
predictions=[]
for i in filename:
  add="/content/test/X/"+i
  a=np.load(add)
  a=r_shape_2(a)
  print(a.shape)
  predictions.append(model.predict(a))
  


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 445ms/step
(1, 64, 1000, 1)
1/1 [==============================] - 0s 16ms/step
(1, 64, 1000, 1)
1/1 [==============================] - 0s 15ms/step
(1, 64, 1000, 1)
1/1 [==============================] - 0s 16ms/step
(1, 64, 1000, 1)
1/1 [==============================] - 0s 18ms/step
(1, 64, 1000, 1)
1/1 [==============================] - 0s 14ms/step
(1, 64, 1000, 1)
1/1 [==============================] - 0s 17ms/step
(1, 64, 1000, 1)
1/1 [==============================] - 0s 24ms/step
(1, 64, 1000, 1)
1/1 [==============================] - 0s 14ms/step
(1, 64, 1000, 1)
1/1 [==============================] - 0s 16ms/step
(1, 64, 1000, 1)
1/1 [==============================] - 0s 16ms/step
(1, 64, 1000, 1)
1/1 [==============================] - 0s 14ms/step
(1, 64, 1000, 1)
1/1 [==============================] - 0s 18ms/step
(1, 64, 1000, 1)
1/1 [==============================] - 0s 18ms/step

In [15]:
predictions=np.array(predictions)
predictions=predictions.reshape((2500,11))

In [16]:
for i in range(len(filename)):

  for z in range(11):

    if predictions[i][z]>=0.45:

      predictions[i][z]=1

    else:

      predictions[i][z]=0


In [ ]:
# from sklearn.metrics import f1_score
# k=f1_score(y_test, y_pred, average=None)
# print(k)
# k=np.mean(k,axis=0)
# print(k)

In [14]:
# del(x_train)
# del(y_train)

In [17]:
events_types = {
    0: 'Alarm_bell_ringing', 
    1: 'Blender', 
    2: 'Cat', 
    3: 'Dishes', 
    4: 'Dog',
    5: 'Electric_shaver_toothbrush', 
    6: 'Frying', 
    7: 'Running_water',
    8: 'Silence', 
    9: 'Speech', 
    10: 'Vacuum_cleaner'
}
final_answers=[]
for i in range (2500):
  active_events=[]
  for z in range(11):
    if predictions[i][z]==1:
      active_events.append(events_types[z])
  final_answers.append(active_events)

In [18]:
final_answers

[['Silence', 'Speech'],
 ['Silence', 'Speech'],
 ['Dog', 'Silence', 'Speech'],
 ['Dishes', 'Frying', 'Speech'],
 ['Speech'],
 ['Silence', 'Speech'],
 ['Silence', 'Speech'],
 ['Silence', 'Speech'],
 ['Speech'],
 ['Silence', 'Speech'],
 ['Silence', 'Speech'],
 ['Electric_shaver_toothbrush', 'Silence', 'Speech'],
 ['Cat', 'Silence', 'Speech'],
 ['Cat', 'Silence', 'Speech'],
 ['Speech'],
 ['Cat', 'Silence', 'Speech'],
 ['Silence', 'Speech'],
 ['Speech', 'Vacuum_cleaner'],
 ['Speech'],
 ['Running_water', 'Speech'],
 ['Silence', 'Speech'],
 ['Blender', 'Silence', 'Speech'],
 ['Cat', 'Silence', 'Speech'],
 ['Cat', 'Dog', 'Silence', 'Speech'],
 ['Silence', 'Speech'],
 ['Silence', 'Speech'],
 ['Cat', 'Silence', 'Speech'],
 ['Speech'],
 ['Blender', 'Silence', 'Speech'],
 ['Running_water', 'Silence', 'Speech'],
 ['Dog', 'Silence', 'Speech'],
 ['Speech'],
 ['Silence', 'Speech'],
 ['Silence', 'Speech'],
 ['Speech'],
 ['Silence', 'Speech'],
 ['Silence', 'Speech'],
 ['Electric_shaver_toothbrush', 'Si

In [19]:
df = pd.DataFrame(list(zip(filename, final_answers)),columns =['filename', 'events'])

In [20]:
df.to_csv('predictions.csv')